In [1]:
import requests
import pprint
import json
import pandas as pd
from pandas.io.json import json_normalize
from datetime import datetime
import os

In [2]:
key = "6eMRkOYgWOwMpHRU%2B8OUaGNyEuTr%2FbM1iGNf5BmXdhhdKFzGuwZ%2FVaQ%2B2yI3%2F%2BTyCvECv7hQzAoRY7U4gzfpEg%3D%3D"

numOfRows = 1000

start_date = "20170101"
end_date = "20230510"

# url = f"http://apis.data.go.kr/1360000/FmlandWthrInfoService/getDayStatistics?serviceKey={key}&numOfRows={numOfRows}&pageNo={pageNo}&ST_YMD={start_date}&ED_YMD={end_date}&AREA_ID=&PA_CROP_SPE_ID={spe_code}&dataType=JSON"

In [3]:
pummok_list = [
    {'pumjongCd_list': ['PA150101'], 'pummok_name': '방울토마토', 'pummok_numb': 1},
    {'pumjongCd_list': ['PA350301'], 'pummok_name': '청양고추', 'pummok_numb': 2},
    {'pumjongCd_list': ['PA350201'], 'pummok_name': '꽈리고추', 'pummok_numb': 3},
    {'pumjongCd_list': ['PA040101'], 'pummok_name': '고구마', 'pummok_numb': 4},
    {'pumjongCd_list': ['PA020101', 'PA020201', 'PA020301'], 'pummok_name': '감자', 'pummok_numb': 5},
    {'pumjongCd_list': ['PA170201','PA170301','PA170401','PA170101'], 'pummok_name': '배추', 'pummok_numb': 6},
    {'pumjongCd_list': ['PA240101','PA240201','PA240301','PA240401'], 'pummok_name': '양배추', 'pummok_numb': 7},
    {'pumjongCd_list': ['PA220101'], 'pummok_name': '시금치', 'pummok_numb': 8},
    {'pumjongCd_list': ['PA280201'], 'pummok_name': '다다기오이', 'pummok_numb': 11},
    {'pumjongCd_list': ['PA280101'], 'pummok_name': '가시오이', 'pummok_numb': 12},
    {'pumjongCd_list': ['PA280301'], 'pummok_name': '취청오이', 'pummok_numb': 13},
    {'pumjongCd_list': ['PA030101'], 'pummok_name': '건고추', 'pummok_numb': 14},
    {'pumjongCd_list': ['PA360101'], 'pummok_name': '애호박', 'pummok_numb': 15},
    {'pumjongCd_list': ['PA360201'], 'pummok_name': '쥬키니호박', 'pummok_numb': 16},
    {'pumjongCd_list': ['PA130201', 'PA130301', 'PA130401', 'PA130101'], 'pummok_name': '무', 'pummok_numb': 18},
    {'pumjongCd_list': ['PA120101','PA120201','PA120301',], 'pummok_name': '깐마늘', 'pummok_numb': 19},
    {'pumjongCd_list': ['PA250101','PA250201'], 'pummok_name': '양파', 'pummok_numb': 20},
    {'pumjongCd_list': ['PA090101','PA090201','PA090301','PA090401'], 'pummok_name': '대파', 'pummok_numb': 21},
    {'pumjongCd_list': ['PA290101'], 'pummok_name': '쪽파', 'pummok_numb': 22},
    {'pumjongCd_list': ['PA140101'], 'pummok_name': '미나리', 'pummok_numb': 23},
    {'pumjongCd_list': ['PA050101'], 'pummok_name': '깻잎', 'pummok_numb': 24},
    {'pumjongCd_list': ['PA260101'], 'pummok_name': '얼갈이배추', 'pummok_numb': 28},
    {'pumjongCd_list': ['PA080101','PA080201','PA080301','PA080401'], 'pummok_name': '당근', 'pummok_numb': 29},
]


In [ ]:
fail_pummok_list = [
    {'pumjongCd_list': ['PA170201','PA170301','PA170401','PA170101'], 'pummok_name': '배추', 'pummok_numb': 6},
    {'pumjongCd_list': ['PA130201', 'PA130301', 'PA130401', 'PA130101'], 'pummok_name': '무', 'pummok_numb': 18},
    {'pumjongCd_list': ['PA120101','PA120201','PA120301',], 'pummok_name': '깐마늘', 'pummok_numb': 19},
    {'pumjongCd_list': ['PA090101','PA090201','PA090301','PA090401'], 'pummok_name': '대파', 'pummok_numb': 21},
]


In [15]:
def filter_items(items, column_names):
    filtered_items = []
    encountered_values = set()

    for item in items:
        # 각 열의 값 모두 연결해서 하나의 key로 만들기
        key = ':'.join([str(item[col]) for col in column_names])
          # 특정 키 값 중복 확인
        if key not in encountered_values:
            filtered_items.append(item)
            encountered_values.add(key)
    return filtered_items

In [16]:
import os

for pummok in pummok_list:
    
    spe_code_list = pummok['pumjongCd_list']
    spe_name = pummok['pummok_name']
    spe_numb = pummok['pummok_numb']

    print(spe_name)

    output_folder = 'excel_result/weather/output'
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    output_file = f'{output_folder}/weather_{spe_numb}_{spe_name}.csv'

    for spe_code in spe_code_list:
        # 첫 페이지 데이터를 가져와 totalCount를 확인
        api_url = f"http://apis.data.go.kr/1360000/FmlandWthrInfoService/getDayStatistics?serviceKey={key}&numOfRows=1&pageNo=1&ST_YMD={start_date}&ED_YMD={end_date}&AREA_ID=999999999&PA_CROP_SPE_ID={spe_code}&dataType=JSON"
        response = requests.get(api_url)
        if response.status_code == 200:
            contents = response.text
            json_ob = json.loads(contents)

            # totalCount 확인
            totalCount = json_ob['response']['body']['totalCount']
        else:
            print(f"Error {response.status}: failed to fetch totalCount for spe_code {spe_code}")
            continue

        pageNo = totalCount // numOfRows if totalCount % numOfRows == 0 else totalCount // numOfRows + 1  # 데이터를 받아오는 횟수

        for page in range(1, pageNo + 1):
            api_url = f"http://apis.data.go.kr/1360000/FmlandWthrInfoService/getDayStatistics?serviceKey={key}&numOfRows={numOfRows}&pageNo={page}&ST_YMD={start_date}&ED_YMD={end_date}&AREA_ID=&PA_CROP_SPE_ID={spe_code}&dataType=JSON"
            response = requests.get(api_url)

            if response.status_code == 200:
                contents = response.text
                json_ob = json.loads(contents)

                # 'body' 키가 있는지 확인
                if 'body' in json_ob['response']:
                    items = json_ob['response']['body']['items']['item']
                    # 데이터 중 특정 조건에 맞는 아이템만 추출
                    filtered_items = filter_items(items, ["dayAvgRhm","dayAvgTa","dayAvgWs","dayMaxTa","dayMinRhm","dayMinTa","daySumRn","daySumSs"])
                    
                    # 데이터프레임 생성
                    df = pd.DataFrame(filtered_items)
                    df = df.rename(columns={"areaId": "지역코드", "areaName": "지역이름", "dayAvgRhm": "평균습도", "dayAvgTa": "평균온도", "dayAvgWs": "평균풍속",
                                            "dayMaxTa": "최고온도", "dayMinRhm": "최저습도", "dayMinTa": "최저온도", "daySumRn": "강수량", "daySumSs": "일조시간",
                                            "paCropName": "품목명", "paCropSpeId": "품목코드", "paCropSpeName": "품종명", "wrnCd": "특보코드", "wrnCount": "특보여부", "ymd": "datadate"})
                    # 문자열을 날짜로 변환
                    df['datedate'] = pd.to_datetime(df['datadate'])
                    # 연, 월, 일을 추출하여 새로운 문자열 생성
                    df['datedate'] = df['datedate'].dt.strftime('%Y%m%d').astype(int)

                    selected_columns = ['datadate','지역코드','지역이름','품목명','품종명','평균온도','최고온도','최저온도','평균습도','최저습도','강수량','평균풍속','일조시간']
                    df = df[selected_columns]

                    if os.path.exists(output_file):
                        df.to_csv(output_file, mode='a', header=False, index=False)
                    else:
                        df.to_csv(output_file, mode='w', header=True, index=False)

                    print(f"{spe_name}: Saved data for {page}/{pageNo}")
                # 더 이상 데이터가 없다면 반복문 종료
                else:
                    print(f"Finished fetching data for spe_code {spe_code} at page {page}")
                    break
            else:
                print(f"Error {response.status}: failed to fetch data for spe_code {spe_code} at page {page}")
                break



쪽파
쪽파: Saved data for 1/11
쪽파: Saved data for 2/11
쪽파: Saved data for 3/11
쪽파: Saved data for 4/11
쪽파: Saved data for 5/11
쪽파: Saved data for 6/11
쪽파: Saved data for 7/11
쪽파: Saved data for 8/11
쪽파: Saved data for 9/11
쪽파: Saved data for 10/11
쪽파: Saved data for 11/11
미나리
미나리: Saved data for 1/34
미나리: Saved data for 2/34
미나리: Saved data for 3/34
미나리: Saved data for 4/34
미나리: Saved data for 5/34
미나리: Saved data for 6/34
미나리: Saved data for 7/34
미나리: Saved data for 8/34
미나리: Saved data for 9/34
미나리: Saved data for 10/34
미나리: Saved data for 11/34
미나리: Saved data for 12/34
미나리: Saved data for 13/34
미나리: Saved data for 14/34
미나리: Saved data for 15/34
미나리: Saved data for 16/34
미나리: Saved data for 17/34
미나리: Saved data for 18/34
미나리: Saved data for 19/34
미나리: Saved data for 20/34
미나리: Saved data for 21/34
미나리: Saved data for 22/34
미나리: Saved data for 23/34
미나리: Saved data for 24/34
미나리: Saved data for 25/34
미나리: Saved data for 26/34
미나리: Saved data for 27/34
미나리: Saved data for 28/34
미나리: Sav

### Data_ 날짜별 정렬

In [ ]:
# input/output directory 경로 설정
input_folder = 'excel_result/weather/output'
output_folder = 'excel_result/weather/output_sorted'

# output_sorted 폴더가 없다면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# input folder에서 모든 csv 파일 가져오기
csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

for csv_file in sorted(csv_files):
    # CSV 파일을 데이터 프레임으로 읽어오기
    df = pd.read_csv(os.path.join(input_folder, csv_file))
    
    # 문자열을 날짜로 변환
    df['datadate'] = pd.to_datetime(df['datadate'])
    # 연, 월, 일을 추출하여 새로운 문자열 생성
    df['datadate'] = df['datadate'].dt.strftime('%Y%m%d').astype(int)

    # datadate 열을 기준으로 정렬하기
    df = df.sort_values('datadate')

    # 정렬된 데이터 프레임을 CSV 파일로 저장하기
    df.to_csv(os.path.join(output_folder, csv_file), index=False)

### Data_ 날짜별 Avg

In [ ]:
# input/output directory 경로 설정
input_folder = 'excel_result/weather/output_sorted'
output_folder = 'excel_result/weather/output_avg'

# output_sorted 폴더가 없다면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# input folder에서 모든 csv 파일 load
csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

for csv_file in sorted(csv_files):
    # CSV 파일을 데이터 프레임으로 read
    df = pd.read_csv(os.path.join(input_folder, csv_file))

    df = df[(df[["평균온도", "최고온도", "최저온도"]] != 0).any(axis=1)]

    # 날짜별로 그룹화하고, 주산지들의 각 날씨 정보 평균내기
    grouped_df = df.groupby('datadate').agg({
        '평균온도': 'mean',
        '최고온도': 'mean',
        '최저온도': 'mean',
        '평균습도': 'mean',
        '최저습도': 'mean',
        '강수량': 'mean',
        '평균풍속': 'mean',
        '일조시간': 'mean'
    }).reset_index()

    print(grouped_df)

    # 결과 CSV 파일로 저장
    output_file = csv_file.replace(".csv", "_averaged.csv")
    grouped_df.to_csv(os.path.join(output_folder, csv_file), index=False)